We scanned *a lot* of teeth for the ZMK guys and gals.
Let's programmatically generate overview images of them.

In [1]:
import platform
import os
import glob
import pandas
# import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
# import seaborn
import dask
import dask_image.imread
from dask.distributed import Client
client = Client()
# Then go to http://localhost:8787/status
# from numcodecs import Blosc
from tqdm import notebook
# import skimage.filters
# import skimage.morphology
# import skimage.segmentation
# import scipy.ndimage

In [2]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:8787/status"


In [3]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit

In [4]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [5]:
# Different locations if running either on Linux or Windows
FastSSD = True
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
else:
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'ZMK')
print('We are loading all the data from %s' % Root)

We are loading all the data from /media/habi/Fast_SSD/ZMK


In [6]:
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})

In [7]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [8]:
# Look only for folders: https://stackoverflow.com/a/38216530
Data['Folder'] = glob.glob(os.path.join(Root, 'ToothBattallion', '*' + os.path.sep))

In [9]:
print('We found %s tooth folders in %s' % (len(Data), Root))

We found 4 tooth folders in /media/habi/Fast_SSD/ZMK


In [10]:
# Look for the relevant logfiles
Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]

In [11]:
Data['Sample'] = [os.path.splitext(os.path.basename(l))[0] for l in Data['LogFile']]

In [12]:
# Proper sorting *with* leading zeros :)
Data.sort_values(by=['Sample'], inplace=True)

In [13]:
# Reconstruction filenames
Data['OutputNameRec'] = [os.path.join(f, sample + '_rec.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [14]:
# Load the reconstructions a zarr arrays
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRec']]

In [15]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [16]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [17]:
# Cropped filenames
Data['OutputNameRecCrop'] = [os.path.join(f, sample + '_rec_crop.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [18]:
# Load the cropped zarr arrays 
CroppedReconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRecCrop']]

In [19]:
# Update the size
Data['Size'] = [rec.shape for rec in CroppedReconstructions]

In [20]:
# Pulpa file names
Data['OutputNamePulpa'] = [os.path.join(f, sample + '_pulpa.zarr') for f, sample in zip(Data.Folder, Data.Sample)]

In [21]:
# Load the cropped zarr arrays back in as *reconstructions*
Pulpas = [dask.array.from_zarr(file) for file in Data['OutputNamePulpa']]

In [22]:
# 3D-Display

In [23]:
import itkwidgets
from itkwidgets import view  # 3d viewer

In [24]:
import ipywebrtc as webrtc
import time

In [25]:
whichone=0
Data.Sample[whichone]

'Tooth045'

In [26]:
TeethPulpa = dask.array.maximum(Pulpas[whichone][1:] * 255,
                                CroppedReconstructions[whichone][1:]).astype('uint8')

In [27]:
tooth = numpy.rot90(TeethPulpa[::25,::25,::25].compute(), k=1)
viewer = view(tooth,
              annotations=False,
              ui_collapsed=True,
              rotate=True,
              gradient_opacity=0.618)
viewer

Viewer(annotations=False, geometries=[], gradient_opacity=0.618, point_sets=[], rendered_image=<itkImagePython…

In [28]:
# https://github.com/InsightSoftwareConsortium/itkwidgets/issues/344#issuecomment-652735527
from IPython.display import display

In [29]:
display(viewer)
image_recorder = webrtc.ImageRecorder(stream=viewer,
                                      autosave=True,
                                      filename=(os.path.join(os.path.sep,
                                                             'home',
                                                             'habi',
                                                             'tmp',
                                                             'screenshot.png')))
image_recorder.recording=True

Viewer(annotations=False, geometries=[], gradient_opacity=0.618, point_sets=[], rendered_image=<itkImagePython…

In [41]:
display(viewer)
recorder = webrtc.VideoRecorder(stream=viewer,
                                autosave=True,
                                filename=(os.path.join(os.path.sep,
                                                       'home',
                                                       'habi',
                                                       'tmp',
                                                       'video.mp4')))
recorder.recording = True
time.sleep(5)
recorder.recording = False
recorder.autosave = False
recorder.download()

In [44]:
display(viewer)
recorder = webrtc.VideoRecorder(stream=viewer,
                                autosave=True,
                                filename=(os.path.join(os.path.sep,
                                                       'home',
                                                       'habi',
                                                       'tmp',
                                                       'video.mp4')))
recorder.recording = True
time.sleep(10)
recorder.recording = False

Viewer(annotations=False, camera=array([[ 124.70138, -113.3729 ,  134.62106],
       [  13.     ,   57.     , …

In [ ]:
# for c, smpl in Data[-5:].iterrows():
#     print(smpl.Sample)
#     viewer = view(CroppedReconstructions[c][::25,::25,::25],
#                   annotations=False,
#                   ui_collapsed=True,
#                   rotate=False,
#                   gradient_opacity=0.618)
#     viewer.camera = numpy.array([[-700, 0,  90],
#                                  [ 90,  0,  90],
#                                  [ 0,  1,  0]], dtype=numpy.float32)    
#     display(viewer)
#     time.sleep(1)    
#     recorder = webrtc.ImageRecorder(stream=viewer,
#                                     filename=os.path.join(Data['Folder'][whichone],
#                                                           Data['Sample'][whichone] + '.3D.Reconstruction.png'),
#                                     autosave=True)
#     recorder
#     recorder.recording = True
#     time.sleep(1)
#     recorder.recording = False    